In [1]:
from collections import defaultdict
import numpy as np
import pandas as pd 
from sklearn.utils import shuffle

In [2]:
# load text

def load_train_data(bow, data_path):
    reviews = []
    f = open(data_path, "r")
    for line in f:
        review = line.split(" ")
        reviews.append(review)
        for word in review:
            if word not in bow:
                bow[word] = len(bow)
            else:
                bow[word] = bow[word] + 1
    return reviews
                
                
def load_validation_data(data_path):
    reviews = []
    f = open(data_path, "r")
    for line in f:
        review = line.split(" ")
        reviews.append(review)
    return reviews
        
def generate_BOW(reviews, bow, length, label):
    raw = []
    for review in reviews:
        cur_bow = np.zeros(length)
        for word in review:
            if word in bow:
                cur_bow[bow[word]] += 1
            else:
                cur_bow[bow['UNKNOWN']] += 1
        raw.append([cur_bow, label])
        
    return raw
                
            
train_decep = "./train/deceptive.txt"
train_truth = "./train/truthful.txt"
validation_decep = "./validation/deceptive.txt"
validation_truth = "./validation/truthful.txt"

bow = {}

deceptive_reviews_train  = load_train_data(bow, train_decep)
truthful_reviews_train = load_train_data(bow, train_truth)
bow['UNKNOWN'] = len(bow)

deceptive_reviews_val = load_validation_data(validation_decep)
truthful_reviews_val = load_validation_data(validation_truth)            
    
length = len(bow)

# generate BOW
raw_train_decep = generate_BOW(deceptive_reviews_train, bow, length, 0)
raw_train_truth = generate_BOW(truthful_reviews_train, bow, length, 1)
raw_train = raw_train_decep + raw_train_truth

raw_val_decep = generate_BOW(deceptive_reviews_val, bow, length, 0)
raw_val_truth = generate_BOW(truthful_reviews_val, bow, length, 1)
raw_val = raw_val_decep + raw_val_truth

train_X_y = shuffle(pd.DataFrame(raw_train, columns = ['X', 'y']))
val_X_y = shuffle(pd.DataFrame(raw_val, columns = ['X', 'y']))


In [3]:
# train NB 
import sklearn
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

X_train, y_train = list(train_X_y.loc[:,'X']), list(train_X_y.loc[:,'y'])
X_val, y_val = list(val_X_y.loc[:,'X']), list(val_X_y.loc[:,'y'])

clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
y_pred = clf.predict(X_val)
# print(clf.predict(X_val)[-100:])

print("Accuracy:", metrics.accuracy_score(y_val, y_pred))

Accuracy: 0.89453125
